# 4. Musculoskeletal Models, Motion Files, and MuscleAnalysis

## 4.1. Objectives

**Introduction**

Musculoskeletal models enable us to study neuromuscular coordination, analyze athletic performance, and estimate musculoskeletal loads [[1]](https://mitpress.mit.edu/9780262044202/biomechanics-of-movement/). In [OpenSim](https://opensim.stanford.edu/) [[2]](https://doi.org/10.1109/TBME.2007.901024), a musculoskeletal model consists of rigid body segments connected by joints. Muscles span these joints and generate forces and motion. Once a musculoskeletal model is created, OpenSim enables users to create custom studies, including investigating the effects of musculoskeletal geometry, joint kinematics, and muscle-tendon properties on the forces and joint moments that the muscles can produce. With OpenSim, our goal is to provide a framework that allows the biomechanics community to create, share, and extend a library of models and dynamic simulation tools that can be used to study and quantify human and animal movement.  

**Purpose**

The purpose of this tutorial is to introduce users to the use of Conda OpenSim for musculoskeletal modeling by investigating how muscle-tendon lengths and moment arms depend on limb configuration. In this tutorial, you will:

*   Become familiar with OpenSim's API.
*   Discover some limitations of musculoskeletal models.
*   Work with motion files.
*   Use the `MuscleAnalysis` tool.
*   Use OpenSim to analyze an important clinical problem.

**Format**

Each section of the tutorial guides you through certain tools within OpenSim and asks you to answer a few questions. The questions can be answered based on information from OpenSim and basic knowledge of the human musculoskeletal system. As you complete each section of the tutorial, feel free to explore OpenSim and the lower extremity model further on your own.


**Resources**

All of the files necessary to complete this tutorial are available in the following Google Drive folder: https://drive.google.com/drive/folders/1n8kmlGEVj2HEexLZr5NlYA6nBW_dggiY?usp=sharing

While you complete the tutorial, you will find cells that automatically download the required files.

To obtain a link to one of the files in Google Drive, right-click over it and click on **Get Link**.

## 4.2. Set up Conda and OpenSim

First, set up the environment by executing the following cell (see [Tutorial 1: Introduction to OpenSim in Colab](https://drive.google.com/file/d/1P_2IRJFzdodS1-ce4BsOsC9d8xWXCqXS/view?usp=sharing) for more information).

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

Don't worry if after executing the previous cell you get an error saying that your session has failed. The reason for this is that condacolab needs to restart the session for the changes to take effect. Because of this, you have to execute the previous cell first, before executing any other cell.

Now, let's install the OpenSim conda package.

In [ ]:
!conda install -c opensim-org opensim

## 4.3. Downloading and exploring the model

In this section, you will download a model of the lower extremity [[3]](https://doi.org/10.1109/10.102791), and then load it into OpenSim. The model represents an adult subject with an approximate height of 1.8 m and an approximate mass of 75 kg. The model consists of 13 rigid body segments and includes the lines of action of 92 muscles (43 per leg and 6 at the torso).

First, let's download the model file (`gait2392.osim`). The file is stored in a Google Drive folder (See **Resources** in the Objectives section).

In [ ]:
!gdown "1Qiqno8NWrBJyo9wMingz1HbJ5FbvfIHo&confirm=t" # gait2392.osim

Now, let's import OpenSim, load the model, and print some metadata.

In [ ]:
import opensim as osim

# Load the model.
gait2392 = osim.Model('gait2392.osim')

# Print metadata.
print("Name of the model:", gait2392.getName())
print("Author:", gait2392.get_credits())
print("Publications:", gait2392.get_publications())
print("Length Unit:", gait2392.get_length_units())
print("Force Unit:", gait2392.get_force_units())
print("Gravity:", gait2392.get_gravity())

Let's explore the coordinates of the model. We know the model has 23 coordinates from the name of the model file.

In [ ]:
# Print the number of coordinates.
print("Num Coordinates:", gait2392.getNumCoordinates())
print()

# For each coordinate, print some information, such as its name or motion type.
for coordinate in gait2392.getCoordinateSet():
  print("  Coordinate Name:", coordinate.getName())
  print("  Coordinate Absolute Path:", coordinate.getAbsolutePathString())
  
  # Motion type is an enumerate (0:Undefined, 1:Rotational, 2:Translational, 3:Coupled).
  motion_type = coordinate.getMotionType()
  motion_type_string = ""
  if motion_type == 0:
    motion_type_string = "Undefined"
  elif motion_type == 1:
    motion_type_string = "Rotational"
  elif motion_type == 2:
    motion_type_string = "Translational "
  elif motion_type == 3:
    motion_type_string = "Coupled "
  print("  Coordinate Motion Type:", motion_type_string)
  
  print()

Let's explore some of the model's joint properties.

In [ ]:
# Print number of joints.
print("Num Joints:", gait2392.getNumJoints())
print()

# For each joint, print some information, such as its name or components.
for joint in gait2392.getJointSet():
  print("Joint Name:", joint.getName())
  print("Joint Absolute Path:", joint.getAbsolutePathString())
  print("Components:")
  for component in joint.getComponentsList():
    print("  Component Name:", component.getName())
    print("  Component Absolute Path:", component.getAbsolutePathString())
  print()

And finally, let's explore the muscles in the model. We know the model has 92 muscles from the name of the model file.



In [ ]:
# Print the number of muscles.
print("Num Muscles:", gait2392.getMuscles().getSize())
print()

for muscle in gait2392.getMuscles():
  print("Muscle Name:", muscle.getName())
  print("Muscle Absolute Path:", muscle.getAbsolutePathString())
  print()

## 4.4. Working with motion files

In this section you will load information from a motion file. A motion file (`.mot`) is a tab separated values (tsv) file containing a header with information about the file (e.g., if the values are in degrees, number of columns), and a set of columns containing how different values in the model evolve over time (e.g., pelvis rotation, pelvis tilt, hip flexion).

Since the motion file already contains the values resulting from a simulation, you will not need to simulate the model.

First, let's download the motion file (`normal_gait.mot`). The file is stored in a Google Drive folder (See **Resources** in the Objectives section).

In [ ]:
!gdown "1hAhf9msXfW0jRau5mIQSGDIBcY_mGZ1K&confirm=t" # normal_gait.mot

Use a `TableProcessor` to load and process the file. Print the column labels to check the file has been processed properly.

In [ ]:
# Use the TableProcessor to read the motion file.
tableTime = osim.TimeSeriesTable('normal_gait.mot')
# Process the file.
# Print the column labels.
print(tableTime.getColumnLabels())

In this example, we will plot the knee angle and hip adduction between both legs. In the next cell, you will extract this information from the motion file.

In [ ]:
# Get columns we want to analyze, and the time column (independent column).
y_knee_angle_r = tableTime.getDependentColumn('knee_angle_r')
y_knee_angle_l = tableTime.getDependentColumn('knee_angle_l')
y_hip_adduction_r = tableTime.getDependentColumn('hip_adduction_r')
y_hip_adduction_l = tableTime.getDependentColumn('hip_adduction_l')
x_time = tableTime.getIndependentColumn()

We can use Seaborn to plot the information.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

sns.set()

fig, axes = plt.subplots(2, 1, figsize=(10, 10))

fig.suptitle('Normal Gait')

# Plot the knee angles on the first subplot.
sns.lineplot(x=x_time, y=y_knee_angle_r.to_numpy(), ax=axes[0], label='knee_angle_r')
sns.lineplot(x=x_time, y=y_knee_angle_l.to_numpy(), ax=axes[0], label='knee_angle_l').set(title='Knee angle', xlabel="Time", ylabel="Knee Angle")

# Plot the hip adductions on the second subplot.
sns.lineplot(x=x_time, y=y_hip_adduction_r.to_numpy(), ax=axes[1], label='hip_adduction_r')
sns.lineplot(x=x_time, y=y_hip_adduction_l.to_numpy(), ax=axes[1], label='hip_adduction_l').set(title='Hip adduction', xlabel="Time", ylabel="Hip Adduction")

# Set the spacing between subplots
plt.subplots_adjust(wspace=0.5, hspace=0.5)

## 4.5. MuscleAnalysis: Assessment of Hamstrings Length During Crouch Gait

In this final section of the tutorial, you will use OpenSim to investigate a possible cause of crouch gait, one of the most common walking abnormalities among individuals with cerebral palsy. It is characterized by excessive flexion of the knee during stance phase, which is often accompanied by exaggerated flexion and internal rotation of the hip. One hypothesized cause of crouch gait is short hamstrings, and orthopedic surgeons will sometimes lengthen the hamstrings of such patients in an attempt to improve their posture and gait. However, other causes of excessive knee flexion are possible (e.g. weak ankle plantar flexors), and lengthening the hamstrings can compromise these muscles' force-generation capabilities [[4]](https://doi.org/10.1016/j.gaitpost.2005.03.003). How can a surgeon determine whether a hamstring lengthening procedure is warranted? 

One possible way to judge whether a patient's hamstrings are shorter than "normal" is to develop a musculoskeletal model and compare the length of the hamstrings during the patient's crouch gait cycle to the length of the hamstrings during a normal gait cycle. Suppose that an orthopaedic surgeon has brought you some kinematic data for a patient who walks with a crouch gait. The surgeon is contemplating whether to operate and wants your opinion.  

### 4.5.1. Range of Motion

First, let's download the crouch gait motion file (`crouch_gait.mot`).The file is stored in a Google Drive folder (See **Resources** in the Objectives section). We already downloaded the normal gait motion file in the previous section.

In [ ]:
!gdown "1p6wDdagbjZ8jbw_zr9mxN_4dnNIm38KB&confirm=t" # crouch_gait.mot

Use a `TableProcessor` to load and process the motion file. Print the column labels to check the files have been processed properly.

In [ ]:
# Use the TableProcessor to read the motion file.
table_processor_normal_gait = osim.TableProcessor('normal_gait.mot')
# Process the file.
table_normal_gait = table_processor_normal_gait.process()
# Print labels for each column.
print(table_normal_gait.getColumnLabels())

# Use the TableProcessor to read the motion file.
table_processor_crouch_gait = osim.TableProcessor('crouch_gait.mot')
# Process the file.
table_crouch_gait = table_processor_crouch_gait.process()
# Print labels for each column.
print(table_crouch_gait.getColumnLabels())

Let's obtain the information we want to plot. This time, we obtain two time columns (independent columns), one per each table. Since both have a range from 0 to 1, we can plot them and compare them.

In [ ]:
# Get columns we want to represent.
normal_gait_knee_angle_r = table_normal_gait.getDependentColumn('knee_angle_r')
crouch_gait_knee_angle_r = table_crouch_gait.getDependentColumn('knee_angle_r')

# Get independent columns of each table (time).
normal_gait_time = table_normal_gait.getIndependentColumn()
crouch_gait_time = table_crouch_gait.getIndependentColumn()

Now let's plot both data series to quantitatively compare.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

sns.set()

sns.lineplot(x=normal_gait_time, y=normal_gait_knee_angle_r.to_numpy(), label='normal_gait_knee_angle_r');
sns.lineplot(x=crouch_gait_time, y=crouch_gait_knee_angle_r.to_numpy(), label='crouch_gait_knee_angle_l').set(title='Normal Gait vs. Crouch Gait - Knee Angle', xlabel='Time', ylabel='Knee Angle');

**Questions: Range of Motion**


**1.** Label the times at which heel strike and toe-off occur, and the stance and swing intervals. 



**2.** What is the range of motion for knee flexion during the stance phase for normal gait?

**3.** How does the knee flexion range of motion for crouch gait compare to that of normal gait?

### 4.5.2. Hamstrings Length

An orthopedic surgeon believes that a patient may benefit from a hamstring lengthening surgery. You are tasked to compare the hamstrings (semitendinosus) length over a patient's crouch gait cycle to the hamstrings length for a "normal" gait cycle. This time, a `MuscleAnalysis` must be performed to obtain the fiber lengths of the muscles.

First, let's load the motion files to obtain the initial and ending time. In both files, the initial and ending times are the same, so we obtain the values from just one of them.

In [ ]:
# Load motion file and extract time.
tableTime = osim.TimeSeriesTable('normal_gait.mot')
x_time = tableTime.getIndependentColumn()

# Get first and last time.
first_time = x_time[0]
last_time = x_time[len(x_time) - 1]

Now, let's create a `MuscleAnalysis` and configure it. This analysis will be applied two times: One for the normal gait and another for the crouch gait.

In [ ]:
# Define a MuscleAnalysis.
muscle_analysis = osim.MuscleAnalysis()

# Set start and end times for the analysis.
muscle_analysis.setStartTime(first_time)
muscle_analysis.setEndTime(last_time)

# Set the muscle of interest (semitendinosus on the right leg).
muscle_list = osim.ArrayStr()
muscle_list.append("semiten_r")
muscle_analysis.setMuscles(muscle_list)

# Configure the analysis.
muscle_analysis.setOn(True)
muscle_analysis.setStepInterval(1)
muscle_analysis.setInDegrees(True)
muscle_analysis.setComputeMoments(True)

For the next step, we need to create an `AnalyzeTool` that, given a model file and a motion file, to perform the requested analysis. We have to create an `AnalyzeTool` for the normal gait and another one for the crouch gait. This may take a few seconds.

In [ ]:
## Normal Gait.

# Create an AnalyzeTool for normal gait.
analyze_tool_normal_gait = osim.AnalyzeTool()
analyze_tool_normal_gait.setName("Muscle_Analysis_Normal_Gait")

# Set the model file and motion file to analyze.
analyze_tool_normal_gait.setModelFilename("gait2392.osim")
analyze_tool_normal_gait.setCoordinatesFileName("normal_gait.mot")

# Add the MuscleAnalysis to the AnalyzeTool.
analyze_tool_normal_gait.updAnalysisSet().cloneAndAppend(muscle_analysis)

# Directory where results are stored.
analyze_tool_normal_gait.setResultsDir("MA_Normal_Gait_Results")

# Configure AnalyzeTool.
analyze_tool_normal_gait.setReplaceForceSet(False)
analyze_tool_normal_gait.setSolveForEquilibrium(True)
analyze_tool_normal_gait.setStartTime(first_time)
analyze_tool_normal_gait.setFinalTime(last_time)

# Print configuration of the AnalyzeTool to an XML file.
analyze_tool_normal_gait.printToXML("Muscle_Analysis_Normal_Gait_AnalyzeTool_setup.xml")

# Load configuration and run the analyses. 
analyze_tool_normal_gait = osim.AnalyzeTool("Muscle_Analysis_Normal_Gait_AnalyzeTool_setup.xml", True)
result_normal_gait = analyze_tool_normal_gait.run()


## Crouch Gait.

# Create an AnalyzeTool for crouch gait.
analyze_tool_crouch_gait = osim.AnalyzeTool()
analyze_tool_crouch_gait.setName("Muscle_Analysis_Crouch_Gait")

# Set the model file and motion file to analyze.
analyze_tool_crouch_gait.setModelFilename("gait2392.osim")
analyze_tool_crouch_gait.setCoordinatesFileName("crouch_gait.mot")

# Add the Muscleanalysis to the AnalyzeTool.
analyze_tool_crouch_gait.updAnalysisSet().cloneAndAppend(muscle_analysis)

# Directory where results are stored.
analyze_tool_crouch_gait.setResultsDir("MA_Crouch_Gait_Results")

# Configure AnalyzeTool.
analyze_tool_crouch_gait.setReplaceForceSet(False)
analyze_tool_crouch_gait.setSolveForEquilibrium(True)
analyze_tool_crouch_gait.setStartTime(first_time)
analyze_tool_crouch_gait.setFinalTime(last_time)

# Print configuration of the AnalyzeTool to an XML file.
analyze_tool_crouch_gait.printToXML("Muscle_Analysis_Crouch_Gait_AnalyzeTool_setup.xml")

# Load configuration and run the analyses. 
analyze_tool_crouch_gait = osim.AnalyzeTool("Muscle_Analysis_Crouch_Gait_AnalyzeTool_setup.xml", True)
result_crouch_gait = analyze_tool_crouch_gait.run()

Now that the analysis has been performed, you can extract the fiber length of the semitendinous muscle!

In [ ]:
## Normal gait

# Get results from file.
table_fiber_length_normal_gait = osim.TimeSeriesTable(
    "MA_Normal_Gait_Results/Muscle_Analysis_Normal_Gait_MuscleAnalysis_FiberLength.sto")

# Get values.
time_normal_gait = table_fiber_length_normal_gait.getIndependentColumn()
fiber_length_semitendinous_normal_gait = table_fiber_length_normal_gait.getDependentColumn("semiten_r")


## Crouch gait

# Get results from file.
table_fiber_length_crouch_gait = osim.TimeSeriesTable(
    "MA_Crouch_Gait_Results/Muscle_Analysis_Crouch_Gait_MuscleAnalysis_FiberLength.sto")

# Get values.
time_crouch_gait = table_fiber_length_crouch_gait.getIndependentColumn()
fiber_length_semitendinous_crouch_gait = table_fiber_length_crouch_gait.getDependentColumn("semiten_r")

Let's create a plot to analyze the fiber length of the semitendinous muscle.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

sns.set()

sns.lineplot(x=time_normal_gait, y=fiber_length_semitendinous_normal_gait.to_numpy(), label='normal_gait_semitendinous_fiber_length')
sns.lineplot(x=time_crouch_gait, y=fiber_length_semitendinous_crouch_gait.to_numpy(), label='crouch_gait_semitendinous_fiber_length').set(title='Normal Gait vs. Crouch Gait - Fiber Length', xlabel='Time', ylabel='Fiber Length');

**Questions: Hamstrings Length**

**1.** Study the curves. Based on the plot, how do the peak hamstring lengths in normal and crouch gait compare? For this patient, would you recommend a hamstrings lengthening surgery? 

**2.** What are some limitations of your analysis?

## 4.6. Conclusion

In this tutorial, you have downloaded and explored a model of the lower extremity. Then, you have used this model to study lower limb kinematics using motion files from previous simulation results. Finally, you have performed a `MuscleAnalysis` to determine the fiber length of the semitendinous muscle to analyze possible underlying causes of crouch gait.

## 4.7. Useful Links





> **OpenSim Website:** https://opensim.stanford.edu/
>
> **OpenSim API Documentation:** https://simtk.org/api_docs/opensim/api_docs/
> 
> **OpenSim Creator Website:** https://opensimcreator.com/
> 
> **SimTK Website:** https://simtk.org/projects/opensim
> 
> **Biomechanics of Movement Course Videos:** https://www.youtube.com/channel/UCDNGy0KKNLQ-ztcL5h2Z6zA

##4.8 Acknowledgments

Allison Arnold and Scott Delp with help from many others.

Thanks to [OpenSimColab](https://simtk.org/projects/opencolab) project [[5]](https://doi.org/10.1080/10255842.2022.2104607) for creating the first OpenSim Conda package.

## 4.9. References

> [1].   Uchida, T. K., Delp, S. L., & Delp, D. (2021). **Biomechanics of movement: The science of sports, robotics, and rehabilitation.** *MIT Press*. https://mitpress.mit.edu/9780262044202/biomechanics-of-movement/
>
> [2].   Delp, S. L., Anderson, F. C., Arnold, A. S., Loan, P., Habib, A., John, C. T., Guendelman, E., & Thelen, D. G. (2007). **OpenSim: open-source software to create and analyze dynamic simulations of movement.** *IEEE Transactions on Bio-Medical Engineering*, 54(11), 1940–1950. https://doi.org/10.1109/TBME.2007.901024
>
> [3].   Delp, S. L., Loan, J. P., Hoy, M. G., Zajac, F. E., Topp, E. L., & Rosen, J. M. (1990). **An interactive graphics-based model of the lower extremity to study orthopaedic surgical procedures.** *IEEE Transactions on Bio-Medical Engineering*, 37(8), 757–767. https://doi.org/10.1109/10.102791
> 
> [4].   Arnold, A. S., Liu, M. Q., Schwartz, M. H., Ounpuu, S., & Delp, S. L. (2006). **The role of estimating muscle-tendon lengths and velocities of the hamstrings in the evaluation and treatment of crouch gait.** *Gait & Posture*, 23(3), 273–281. https://doi.org/10.1016/j.gaitpost.2005.03.003
>
> [5] Mokhtarzadeh, H., Jiang, F., Zhao, S., & Malekipour, F. (2022). **OpenColab project: OpenSim in Google colaboratory to explore biomechanics on the web.** *Computer Methods in Biomechanics and Biomedical Engineering*, 1–9. https://doi.org/10.1080/10255842.2022.2104607